In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import glob
import MeCab

In [2]:
path = "moviedata2/*"
files = glob.glob(path)

In [9]:
#tfidf値合計値出す
#出現数上位10個の単語をリストで取得する関数
def getmax_list(dataframe, id_list, topnum=10, getmin=False):
    s = min(id_list)
    e = max(id_list) + 1
    sum_word = dataframe[s:e].sum(axis=0)
    out = sum_word.nsmallest(topnum) if getmin else sum_word.nlargest(topnum)
    word = out.index.tolist()
    return word
    #return sum_word_df

In [3]:
def concat_df(main_df, sub_df):
    df_list = [main_df, sub_df]
    df_concat = pd.concat(df_list, ignore_index=True)
    return df_concat

In [4]:
import module as md
def make_df(file_path):
    #ファイルパスからデータフレーム
    df = pd.read_json(file_path, orient='records', lines=True)
    #コメント分割
    df = md.sep_comments(df)
    #カラム追加
    df['id'] = file_path
    df["content"] = df["content"].apply(md.preprocessor)
    cd = md.CleaningData(df, 'content')
    df = cd.cleaning()
    df["content"] = df["content"].apply(md.wakati_m_2)
    return df

In [5]:
def join_df(file_list):
    main_df=pd.DataFrame(columns=["content", "id"])
    sub_df=0
    for l in file_list:
        sub_df = make_df(l)
        main_df = concat_df(main_df, sub_df)
    return main_df

In [6]:
df = join_df(files)

In [7]:
#bow
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count = CountVectorizer()
bag = count.fit_transform(df["content"]) #時間でコメント分割
bow_df = pd.DataFrame(bag.toarray(), columns=count.get_feature_names())
#tf-idf
tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True)
#np.set_printoptions(precision=2)#小数点以下二桁
word_tfidf = tfidf.fit_transform(bag.toarray())
tfidf_df = pd.DataFrame(word_tfidf.toarray(), columns=count.get_feature_names())

In [10]:
#各動画の特徴語20個抽出
movie_df=pd.DataFrame(columns=["id", "content"])
for i in range(len(files)):
    movie_Index = df[df['id']==files[i]].index.tolist()
    word = getmax_list(tfidf_df, movie_Index, topnum=20)
    #text = " ".join(word)
    movie_df.loc[i]=[files[i], word]

/usr/local/lib/python3.8/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [19]:
#df保存
#リストを文字列に変換
for i in range(len(movie_df)):
    movie_df["content"].iloc[i] = " ".join(movie_df["content"].iloc[i])
movie_df.to_csv("movie3_df_list.csv")
movie_df

,id,content
0,moviedata2/sm17961470.jsonl,きくお 幸福 鬱くしい 好き 地獄 天国 綺麗 歌詞 ミク ダブステ 中毒 サビ 世界観 ダ...
1,moviedata2/sm12419220.jsonl,シンジ もこう ポケモン 厳選 先生 廃人 ゾロア ポカブ つり マジレス 孵化 うそなき ...
2,moviedata2/sm5430176.jsonl,アリス レン 不思議 番目 おk リン ミク カイト 真っ赤 一時停止 メイコ 薔薇 kai...
3,moviedata2/so15281450.jsonl,アイマス ナンバーワン うたプリ 美希 ここ 周年 公式 op akb ラブライブ 職人 み...
4,moviedata2/sm2069100.jsonl,勉強 テスト 明日 集中 休憩 数学 レポート 時間 今日 みんな 英語 あと 課題 徹夜 ...
...,...,...
495,moviedata2/sm5408420.jsonl,ルル ルルーシュ 太平洋 ひで ギアス ローゼン 触手 スザク みのりん シスター おまw ...
496,moviedata2/sm10116060.jsonl,美琴 黒子 俺の嫁 職人 木山 初春 レールガン 禁書 上条さん そげぶ 原木 ゥゥン りっ...
497,moviedata2/sm20715461.jsonl,サシャ 巨人 エレン ベルトルト ミーナ フラグ 駆逐 ネタバレ 駆逐系男子 画伯 トーマス...
498,moviedata2/sm3120540.jsonl,一番 シャーリー ユフィ ルル 解読不能 モザイクカケラ ルルーシュ ギアス ピカレスク c...
